In [1]:
import os
import sys
if os.getenv("ORCA_PATH"):
    ORCA_PATH = os.getenv("ORCA_PATH")
else:
    # change this to the right path if you use a different path
    # or specify the ORCA_PATH environmental variable
    ORCA_PATH = "../orca"
sys.path.append(ORCA_PATH)

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import orca_predict
orca_predict.load_resources(models=['256M'],use_cuda=True)
from orca_predict import *

In <PYTHONPATH>/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In <PYTHONPATH>/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In <PYTHONPATH>/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [4]:
from selene_utils2 import *
sampler_h1esc = RandomPositionsSamplerHiC(
                 reference_sequence = MemmapGenome(ORCA_PATH+'/resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa', memmapfile='./resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa.mmap'),
                 target= Genomic2DFeatures([ORCA_PATH+'/resources/4DNFI9GMP2J8.rebinned.mcool::/resolutions/32000'],
                           ['r8000'],
                          (8000,8000),
                          cg=True),
                 features = ['r4000'],
                 test_holdout=['chr8', 'chr9'],
                 validation_holdout= ['chr10'],
                 sequence_length= 256000000,
                 position_resolution=32000,
                 random_shift=0,
                 random_strand=False,
                 cross_chromosome=True,
                 permute_segments=False,
                 background_cis_file=ORCA_PATH+'/resources/4DNFI9GMP2J8.rebinned.mcool.expected.res32000.mono.npy',
                 background_trans_file=ORCA_PATH+'/resources/4DNFI9GMP2J8.rebinned.mcool.expected.res32000.trans.npy'
 )
    
sampler_hff = RandomPositionsSamplerHiC(
                 reference_sequence = MemmapGenome(ORCA_PATH+'/resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa', memmapfile='./resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa.mmap'),
                 target= Genomic2DFeatures([ORCA_PATH+'/resources/4DNFI643OYP9.rebinned.mcool::/resolutions/32000'],
                           ['r8000'],
                          (8000,8000),
                          cg=True),
                 features = ['r4000'],
                 test_holdout=['chr8', 'chr9'],
                 validation_holdout= ['chr10'],
                 sequence_length= 256000000,
                 position_resolution=32000,
                 random_shift=0,
                 random_strand=False,
                 cross_chromosome=True,
                 permute_segments=False,
                 background_cis_file=ORCA_PATH+'/resources/4DNFI643OYP9.rebinned.mcool.expected.res32000.mono.npy',
                 background_trans_file=ORCA_PATH+'/resources/4DNFI643OYP9.rebinned.mcool.expected.res32000.trans.npy'
 )

In [5]:
#32-256Mb model evaluation, intrachromosomal interactions
import torch
with torch.no_grad():
    
    mallpreds = []
    malltargets = []
    malltargetnans = []

    for chrm in ['chr9','chr10']:
        chrlen = [len for c, len in hg38.get_chr_lens() if c == chrm][0]
        chrlen_round = chrlen - chrlen % 32000
        
        sequences, targets_h1esc, normmats_h1esc = sampler_h1esc._retrieve_multi(*list(zip([chrm, 0, chrlen_round],['chr1',0, 256000000-chrlen_round])))
        _, targets_hff, normmats_hff = sampler_hff._retrieve_multi(*list(zip([chrm, 0, chrlen_round],['chr1',0, 256000000-chrlen_round])))

        sequences = np.vstack(sequences)
        targets_h1esc = np.vstack([np.hstack(l) for l in targets_h1esc])
        normmats_h1esc = np.vstack([np.hstack(l) for l in normmats_h1esc])
        targets_hff = np.vstack([np.hstack(l) for l in targets_hff])
        normmats_hff = np.vstack([np.hstack(l) for l in normmats_hff])
        
        binstarts = []
        for level in [32, 64, 128, 256]:
             binstarts.append(np.arange(0, (chrlen_round - level * 1000000) / 32000 , 5120000/32000))

        for binstart in binstarts[0]:
            allpreds = []
            alltargets = []
            alltargetnans = []
            seqs =  [torch.FloatTensor(sequences[None,:,:]), torch.FloatTensor(sequences[None,::-1,::-1].copy())]

            twotargets = [ targets_h1esc[None,:], targets_hff[None,:]]
            twonormmats = [ normmats_h1esc[None,:], normmats_hff[None,:]]
            for iii, seq in enumerate(seqs):
                for ii, model in enumerate([h1esc_256m, hff_256m]):
                    target = twotargets[ii]
                    normmat = twonormmats[ii]
                    encoding0 = model.net1(model.net0(seq.cuda().transpose(1,2)))[-1]
                    encoding32, encoding64, encoding128, encoding256  = model.net(encoding0)
                    encodings =  {32:encoding32, 64:encoding64, 128:encoding128, 256:encoding256}
                    def eval_step(model, level, start, target,  normmat, coarse_pred=None):
                        d = int(level/8)
                        target_r = np.nanmean(np.nanmean(np.reshape(target[:,start:start+250* d,start:start+250*d],(target.shape[0],250,d,250,d)),axis=4),axis=2)
                        target_nan =  np.mean(np.mean(np.isnan(np.reshape(target[:,start:start+250* d,start:start+250*d],(target.shape[0],250,d,250,d))),axis=4),axis=2)


                        normmat_nan = np.isnan(normmat)
                        if np.any(normmat_nan):
                            normmat[normmat_nan] = np.nanmin(normmat[~normmat_nan])
                        normmat_r = np.mean(np.mean(np.reshape(normmat[:,start:start+250*d,start:start+250*d],(normmat.shape[0],250,d,250,d)),axis=4),axis=2)

                        if coarse_pred is not None:
                            pred = model.denets[level].forward(encodings[level][:,:,int(start/d):int(start/d)+250], torch.log(torch.Tensor(normmat_r).cuda())[:,None,:,:], coarse_pred)
                        else:
                            pred = model.denets[level].forward(encodings[level][:,:,int(start/d):int(start/d)+250], torch.log(torch.Tensor(normmat_r).cuda())[:,None,:,:])
                        eps = np.nanmin(normmat_r)
                        target_cuda = torch.Tensor( np.log(((target_r+eps)/(normmat_r+ eps)))[:,0:,0:]).cuda()

                        target_np = np.log((eps+target_r)/(eps+normmat_r))[:,0:,0:]

                        return pred, target_np, target_nan
                    
                    if iii == 0:
                        start = 0
                        pred256, target256, targetnan256 = eval_step(model, 256, start, target, normmat)
                        start = int(binstart)
                        pred128, target128, targetnan128 = eval_step(model, 128, start, target, normmat, pred256[:,:,int(binstart/32):int(binstart/32)+125,int(binstart/32):int(binstart/32)+125])
                        start = int(binstart)
                        pred64, target64, targetnan64 = eval_step(model, 64, start, target, normmat, pred128[:,:,:125,:125])
                        start = int(binstart)
                        pred32, target32, targetnan32 = eval_step(model, 32, start, target, normmat, pred64[:,:,:125,:125])
                    else:
                        start = 0
                        pred256, target256, targetnan256 = eval_step(model, 256, start, target[:,::-1,::-1], normmat[:,::-1,::-1])
                        start = 8000 - int(binstart) - 4000
                        pred128, target128, targetnan128 = eval_step(model, 128, start, target[:,::-1,::-1], normmat[:,::-1,::-1], pred256[:,:, 125-int(binstart/32): 250-int(binstart/32), 125-int(binstart/32): 250-int(binstart/32)])
                        start = 8000 - int(binstart) - 2000
                        pred64, target64, targetnan64 = eval_step(model, 64, start, target[:,::-1,::-1], normmat[:,::-1,::-1], pred128[:,:,125:,125:])
                        start = 8000 - int(binstart) - 1000
                        pred32, target32, targetnan32 = eval_step(model, 32, start, target[:,::-1,::-1], normmat[:,::-1,::-1], pred64[:,:,125:,125:])


                    if binstart != 0:
                        pred256 = None
                        target256 = None
                        targetnan256 = None
                    else:
                        if iii == 0:
                            pred256[0,0,int(chrlen_round/1024000):,:]=np.nan
                            target256[0,int(chrlen_round/1024000):,:]=np.nan
                            targetnan256[0,int(chrlen_round/1024000):,:]=np.nan
                            pred256[0,0,:,int(chrlen_round/1024000):]=np.nan
                            target256[0,:,int(chrlen_round/1024000):]=np.nan
                            targetnan256[0,:,int(chrlen_round/1024000):]=np.nan
                        else:
                            pred256[0,0,:250-int(chrlen_round/1024000),:]=np.nan
                            target256[0,:250-int(chrlen_round/1024000),:]=np.nan
                            targetnan256[0,:250-int(chrlen_round/1024000),:]=np.nan
                            pred256[0,0,:,:250-int(chrlen_round/1024000)]=np.nan
                            target256[0,:,:250-int(chrlen_round/1024000)]=np.nan
                            targetnan256[0,:,:250-int(chrlen_round/1024000)]=np.nan
                    if not binstart in binstarts[1]:
                        pred64 = None
                        target64 = None
                        targetnan64 = None
                        
                    if not binstart in binstarts[2]:
                        pred128 = None
                        target128 = None
                        targetnan128 = None
                        

                    allpreds.append([pred32, pred64, pred128, pred256])
                    alltargets.append([target32, target64, target128, target256])
                    alltargetnans.append([targetnan32, targetnan64, targetnan128, targetnan256])

            mallpreds.append(allpreds)
            malltargets.append(alltargets)
            malltargetnans.append(alltargetnans)


<PYTHONPATH>/lib/python3.7/site-packages/cooltools/lib/numutils.py:1317: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur
<PYTHONPATH>/lib/python3.7/site-packages/cooltools/lib/numutils.py:1317: RuntimeWarning: divide by zero encountered in true_divide
  val_cur = ar_cur / armask_cur
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
<PYTHONPATH>/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [6]:
torch.save(mallpreds, './figure_data/mallpreds.256m.pth')
torch.save(malltargets, './figure_data/malltargets.256m.pth')
torch.save(malltargetnans, './figure_data/malltargetnans.256m.pth')

In [7]:
#load precomputed 
import torch
malltargets = torch.load('./figure_data/malltargets.256m.pth')
malltargetnans = torch.load( './figure_data/malltargetnans.256m.pth')
mallpreds = torch.load( './figure_data/mallpreds.256m.pth')

In [8]:
from scipy.stats import pearsonr, spearmanr
from collections import defaultdict
import numpy as np


p1s = defaultdict(list)
p2s = defaultdict(list)
t1s = defaultdict(list)
t2s = defaultdict(list)

levelsn = [32, 64, 128, 256]

for i in range(len(mallpreds)):
    for j in range(4):
        if mallpreds[i][0][j] is not None:
            
            p1 = mallpreds[i][0][j].detach().cpu().numpy()[0,0,:,:][:,:].flatten()+mallpreds[i][2][j].detach().cpu().numpy()[0,0,::-1,::-1][:,:].flatten()
            t1 = malltargets[i][0][j][0,:].reshape((250,250))[:,:].flatten()
            tn1 = malltargetnans[i][0][j][0,:].reshape((250,250))[:,:].flatten()
            p2 = mallpreds[i][1][j].detach().cpu().numpy()[0,0,:,:][:,:].flatten()+mallpreds[i][3][j].detach().cpu().numpy()[0,0,::-1,::-1][:,:].flatten()
            t2 = malltargets[i][1][j][0,:].reshape((250,250))[:,:].flatten()
            tn2 = malltargetnans[i][1][j][0,:].reshape((250,250))[:,:].flatten()

            valid = np.isfinite(t1) * np.isfinite(t2)  * (tn1<=0.25) * (tn2<=0.25) 


            p1[~valid]=np.nan
            t1[~valid]=np.nan

            p1s[j].append(p1)
            t1s[j].append(t1)


            p2[~valid]=np.nan
            t2[~valid]=np.nan
            p2s[j].append(p2)
            t2s[j].append(t2)


<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in less_equal


In [9]:
#Intrachromosal performance
for i, level in enumerate(['32Mb','64Mb','128Mb','256Mb']):
    valid = np.isfinite(t1s[i])  
    print('H1-ESC', level, pearsonr(np.array(p1s[i])[valid], np.array(t1s[i])[valid])[0])
    
for i, level in enumerate(['32Mb','64Mb','128Mb','256Mb']):
    valid = np.isfinite(t2s[i])  
    print('HFF', level, pearsonr(np.array(p2s[i])[valid], np.array(t2s[i])[valid])[0])

for i, level in enumerate(['32Mb','64Mb','128Mb','256Mb']):
    valid = np.isfinite(t2s[i]) 
    print('Diff', level, pearsonr(np.array(p2s[i])[valid]-np.array(p1s[i])[valid], np.array(t2s[i])[valid]-np.array(t1s[i])[valid])[0])

H1-ESC 32Mb 0.8215610872990484
H1-ESC 64Mb 0.8455151960421428
H1-ESC 128Mb 0.8401351029384021
H1-ESC 256Mb 0.8457584284328606
HFF 32Mb 0.7254492726664794
HFF 64Mb 0.7255609148257045
HFF 128Mb 0.7295147325253019
HFF 256Mb 0.7484020607503034
Diff 32Mb 0.4779782642676464
Diff 64Mb 0.5124816993251585
Diff 128Mb 0.5666834241982133
Diff 256Mb 0.6254152099181652


In [10]:
from plotnine import *
import pandas as pd
import plotnine
plotnine.options.figure_size = (8, 6)


np.random.seed(0)
levels = ['32Mb','64Mb','128Mb','256Mb'] 
plotdata = []
for i in range(0,4):
    valid = np.isfinite(t1s[i]) #* np.isfinite(t2s[i])
    subsetind = np.random.permutation(np.sum(valid))[:100000]
    plotdata.append(pd.DataFrame({'x':np.array(p1s[i])[valid][subsetind], 'y':np.array(t1s[i])[valid][subsetind], \
                                  'level': levels[i], 'cell': 'H1-ESC'}))
    plotdata.append(pd.DataFrame({'x':np.array(p2s[i])[valid][subsetind], 'y':np.array(t2s[i])[valid][subsetind], \
                                  'level': levels[i], 'cell': 'HFF'}))
    
plotdata = pd.concat(plotdata, axis=0)
plotdata['level'] = pd.Categorical(plotdata['level'], levels)

plotdata.to_csv('./figure_data/predeval.256m.csv')

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pandas as pd
levels = ['32Mb','64Mb','128Mb','256Mb']
plotdata = []
for i in range(0, 4):
    valid = np.isfinite(t2s[i])  
    thresh = np.quantile(np.abs(np.array(t2s[i])-np.array(t1s[i]))[valid], 0.99)
    valid = valid * (np.abs(np.array(t2s[i])-np.array(t1s[i]))> thresh)
    
    print(np.sum(valid),roc_auc_score(
                 (np.array(t2s[i])[valid]-np.array(t1s[i])[valid])>0,
                 np.array(p2s[i])[valid] - np.array(p1s[i])[valid]))
    fpr, tpr, _ = roc_curve(
                 (np.array(t2s[i])[valid]-np.array(t1s[i])[valid])>0,
                 np.array(p2s[i])[valid] - np.array(p1s[i])[valid])
    plotdata.append(pd.DataFrame({'FPR':fpr,'TPR':tpr,'level':levels[i]}))

plotdata=pd.concat(plotdata, axis=0)
plotdata.to_csv('./figure_data/predeval.roc.256m.csv')


<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


19298 0.9656880226581053
12120 0.9521985547191025
2226 0.9621304642822526
264 0.9960973370064279


<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in greater
  if __name__ == '__main__':


In [12]:
from selene_utils2 import * 
sampler_h1esc = RandomPositionsSamplerHiC(

                 reference_sequence = MemmapGenome(ORCA_PATH+'/resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa', memmapfile=ORCA_PATH+'/resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa.mmap'),
                 target= Genomic2DFeatures([ORCA_PATH+'/resources/4DNFI9GMP2J8.rebinned.mcool::/resolutions/32000'],
                           ['r8000'],
                          (8000,8000),
                          cg=True),
                 features = ['r4000'],
                 test_holdout=['chr9', 'chr10'],
                 validation_holdout= ['chr8'],
                 sequence_length= 256000000,
                 position_resolution=32000,
                 random_shift=0,
                 random_strand=True,
                 cross_chromosome=True,
                 permute_segments=True,
                 length_schedule=[1, [64000000, 128000000]],
                 background_cis_file=ORCA_PATH+'/resources/4DNFI9GMP2J8.rebinned.mcool.expected.res32000.mono.npy',
                 background_trans_file=ORCA_PATH+'/resources/4DNFI9GMP2J8.rebinned.mcool.expected.res32000.trans.npy'
 )
    
sampler_hff = RandomPositionsSamplerHiC(

                 reference_sequence = MemmapGenome(ORCA_PATH+'/resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa', memmapfile=ORCA_PATH+'/resources/Homo_sapiens.GRCh38.dna.primary_assembly.fa.mmap'),
                 target= Genomic2DFeatures([ORCA_PATH+'/resources/4DNFI643OYP9.rebinned.mcool::/resolutions/32000'],
                           ['r8000'],
                          (8000,8000),
                          cg=True),
                 features = ['r4000'],
                 test_holdout=['chr9', 'chr10'],
                 validation_holdout= ['chr8'],
                 sequence_length= 256000000,
                 position_resolution=32000,
                 random_shift=0,
                 random_strand=True,
                 cross_chromosome=True,
                 permute_segments=True,
                 length_schedule=[1, [64000000, 128000000]],
                 background_cis_file=ORCA_PATH+'/resources/4DNFI643OYP9.rebinned.mcool.expected.res32000.mono.npy',
                 background_trans_file=ORCA_PATH+'/resources/4DNFI643OYP9.rebinned.mcool.expected.res32000.trans.npy'
 )

In [13]:
#32-256Mb model evaluation, interchromosomal interactions

import torch
np.random.seed(213)
with torch.no_grad():
    
    callpreds = []
    calltargets = []
    calltargetnans = []
    callnormmats = []
    for i in range(100):
        sampler_h1esc.mode='test'
        sampled = sampler_h1esc.sample(coordinate_only=True)
        sequences, targets_h1esc, normmats_h1esc = sampler_h1esc._retrieve_multi(*sampled[0])
        _, targets_hff, normmats_hff = sampler_hff._retrieve_multi(*sampled[0])

        sequences = np.vstack(sequences)
        targets_h1esc = np.vstack([np.hstack(l) for l in targets_h1esc])
        normmats_h1esc = np.vstack([np.hstack(l) for l in normmats_h1esc])
        targets_hff = np.vstack([np.hstack(l) for l in targets_hff])
        normmats_hff = np.vstack([np.hstack(l) for l in normmats_hff])
        

        allpreds = []
        alltargets = []
        alltargetnans = []
        allnormmats = []
        seqs =  [torch.FloatTensor(sequences[None,:,:]), torch.FloatTensor(sequences[None,::-1,::-1].copy())]

        twotargets = [ targets_h1esc[None,:], targets_hff[None,:]]
        twonormmats = [ normmats_h1esc[None,:], normmats_hff[None,:]]
        for iii, seq in enumerate(seqs):
            for ii, model in enumerate([h1esc_256m, hff_256m]):
                target = twotargets[ii]
                normmat = twonormmats[ii]
                encoding0 = model.net1(model.net0(seq.cuda().transpose(1,2)))[-1]
                encoding32, encoding64, encoding128, encoding256  = model.net(encoding0)
                encodings =  {32:encoding32, 64:encoding64, 128:encoding128, 256:encoding256}
                def eval_step(model, level, start, target,  normmat, coarse_pred=None):
                    d = int(level/8)
                    target_r = np.nanmean(np.nanmean(np.reshape(target[:,start:start+250* d,start:start+250*d],(target.shape[0],250,d,250,d)),axis=4),axis=2)
                    target_nan =  np.mean(np.mean(np.isnan(np.reshape(target[:,start:start+250* d,start:start+250*d],(target.shape[0],250,d,250,d))),axis=4),axis=2)


                    normmat_nan = np.isnan(normmat)
                    if np.any(normmat_nan):
                        normmat[normmat_nan] = np.nanmin(normmat[~normmat_nan])
                    normmat_r = np.mean(np.mean(np.reshape(normmat[:,start:start+250*d,start:start+250*d],(normmat.shape[0],250,d,250,d)),axis=4),axis=2)

                    if coarse_pred is not None:
                        pred = model.denets[level].forward(encodings[level][:,:,int(start/d):int(start/d)+250], torch.log(torch.Tensor(normmat_r).cuda())[:,None,:,:], coarse_pred)
                    else:
                        pred = model.denets[level].forward(encodings[level][:,:,int(start/d):int(start/d)+250], torch.log(torch.Tensor(normmat_r).cuda())[:,None,:,:])
                    eps = np.nanmin(normmat_r)
                    target_cuda = torch.Tensor( np.log(((target_r+eps)/(normmat_r+ eps)))[:,0:,0:]).cuda()

                    target_np = np.log((eps+target_r)/(eps+normmat_r))[:,0:,0:]

                    return pred, target_np, target_nan, normmat_r

                if iii == 0:
                    start = 0
                    pred256, target256, targetnan256, normmat256 = eval_step(model, 256, start, target, normmat)
                    start = 63 * 32
                    pred128, target128, targetnan128, normmat128 = eval_step(model, 128, start, target, normmat, pred256[:,:,63:63+125,63:63+125])
                    start += 62 * 16
                    pred64, target64, targetnan64, normmat64 = eval_step(model, 64, start, target, normmat, pred128[:,:,62:62+125,62:62+125])
                    start += 62 * 8
                    pred32, target32, targetnan32, normmat32 = eval_step(model, 32, start, target, normmat, pred64[:,:,62:62+125,62:62+125])
                else:
                    start = 0
                    pred256, target256, targetnan256, normmat256 = eval_step(model, 256, start, target[:,::-1,::-1], normmat[:,::-1,::-1])
                    start = 62 * 32
                    pred128, target128, targetnan128, normmat128 = eval_step(model, 128, start, target[:,::-1,::-1], normmat[:,::-1,::-1], pred256[:,:,62:62+125,62:62+125])
                    start += 63 * 16
                    pred64, target64, targetnan64, normmat64 = eval_step(model, 64, start, target[:,::-1,::-1], normmat[:,::-1,::-1], pred128[:,:,63:63+125,63:63+125])
                    start += 63 * 8
                    pred32, target32, targetnan32, normmat32 = eval_step(model, 32, start, target[:,::-1,::-1], normmat[:,::-1,::-1], pred64[:,:,63:63+125,63:63+125])

                allpreds.append([pred32, pred64, pred128, pred256])
                alltargets.append([target32, target64, target128, target256])
                alltargetnans.append([targetnan32, targetnan64, targetnan128, targetnan256])
                allnormmats.append([normmat32, normmat64, normmat128, normmat256])

        callpreds.append(allpreds)
        calltargets.append(alltargets)
        calltargetnans.append(alltargetnans)
        callnormmats.append(allnormmats)


<PYTHONPATH>/lib/python3.7/site-packages/cooltools/lib/numutils.py:1317: RuntimeWarning: invalid value encountered in true_divide
  val_cur = ar_cur / armask_cur
<PYTHONPATH>/lib/python3.7/site-packages/cooltools/lib/numutils.py:1317: RuntimeWarning: divide by zero encountered in true_divide
  val_cur = ar_cur / armask_cur
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice
<PYTHONPATH>/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [14]:
torch.save(calltargets, './figure_data/calltargets.256m.pth')
torch.save(calltargetnans, './figure_data/calltargetnans.256m.pth')
torch.save(callnormmats, './figure_data/callnormmats.256m.pth')
torch.save(callpreds, './figure_data/callpreds.256m.pth')

In [15]:
calltargets = torch.load('./figure_data/calltargets.256m.pth')
calltargetnans = torch.load('./figure_data/calltargetnans.256m.pth')
callnormmats = torch.load('./figure_data/callnormmats.256m.pth')
callpreds = torch.load('./figure_data/callpreds.256m.pth')

In [16]:
from scipy.stats import pearsonr, spearmanr
from collections import defaultdict
import numpy as np
normmats = h1esc_256m.background_cis[np.abs(np.arange(8000)[None,:]-np.arange(8000)[:,None])]
normmats = [np.nanmean(np.nanmean(np.reshape(normmats,(2000,4,2000,4)),axis=3),axis=1)[:250,:250],
            np.nanmean(np.nanmean(np.reshape(normmats,(1000,8,1000,8)),axis=3),axis=1)[:250,:250],
            np.nanmean(np.nanmean(np.reshape(normmats,(500,16,500,16)),axis=3),axis=1)[:250,:250],
            np.nanmean(np.nanmean(np.reshape(normmats,(250,32,250,32)),axis=3),axis=1)[:250,:250]]


p1s = defaultdict(list)
p2s = defaultdict(list)
t1s = defaultdict(list)
t2s = defaultdict(list)

levelsn = [32, 64, 128, 256]

for i in range(len(callpreds)):
    for j in range(4):
        if callpreds[i][0][j] is not None:
            
            p1 = callpreds[i][0][j].detach().cpu().numpy()[0,0,:,:][:,:].flatten()+callpreds[i][2][j].detach().cpu().numpy()[0,0,::-1,::-1][:,:].flatten()
            t1 = calltargets[i][0][j][0,:].reshape((250,250))[:,:].flatten()
            tn1 = calltargetnans[i][0][j][0,:].reshape((250,250))[:,:].flatten()
            p2 = callpreds[i][1][j].detach().cpu().numpy()[0,0,:,:][:,:].flatten()+callpreds[i][3][j].detach().cpu().numpy()[0,0,::-1,::-1][:,:].flatten()
            t2 = calltargets[i][1][j][0,:].reshape((250,250))[:,:].flatten()
            tn2 = calltargetnans[i][1][j][0,:].reshape((250,250))[:,:].flatten()

            
            #selecting interchromosomal interactions 
            valid = np.isfinite(t1) * np.isfinite(t2)  * (tn1<=0.25)  * (np.isclose(callnormmats[i][0][j][0], 3.4847237202484393e-06).flatten())


            p1[~valid]=np.nan
            t1[~valid]=np.nan

            p1s[j].append(p1)
            t1s[j].append(t1)

            p2[~valid]=np.nan
            t2[~valid]=np.nan
            p2s[j].append(p2)
            t2s[j].append(t2)


<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: Mean of empty slice
  import sys
<PYTHONPATH>/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice
  


In [17]:
#Interchromosomal performance 
for i, level in enumerate(['32Mb','64Mb','128Mb','256Mb']):
    valid = np.isfinite(t1s[i])  
    print('H1ESC', level, pearsonr(np.array(p1s[i])[valid], np.array(t1s[i])[valid])[0])
    
for i, level in enumerate(['32Mb','64Mb','128Mb','256Mb']):
    valid = np.isfinite(t2s[i])  
    print('HFF', level, pearsonr(np.array(p2s[i])[valid], np.array(t2s[i])[valid])[0])

for i, level in enumerate(['32Mb','64Mb','128Mb','256Mb']):
    valid = np.isfinite(t2s[i]) 
    print('Diff', level, pearsonr(np.array(p2s[i])[valid]-np.array(p1s[i])[valid], np.array(t2s[i])[valid]-np.array(t1s[i])[valid])[0])

H1ESC 32Mb 0.3869214518155263
H1ESC 64Mb 0.5209612660142798
H1ESC 128Mb 0.6377317985850213
H1ESC 256Mb 0.7373308957499379
HFF 32Mb 0.357238538502456
HFF 64Mb 0.47171294377137646
HFF 128Mb 0.6071246770073321
HFF 256Mb 0.7108015679685028
Diff 32Mb 0.18144108039893062
Diff 64Mb 0.21590998331714942
Diff 128Mb 0.27160963844701286
Diff 256Mb 0.3723438985789733


In [18]:
from plotnine import *
import pandas as pd
import plotnine
plotnine.options.figure_size = (8, 6)


np.random.seed(0)
levels = ['32Mb','64Mb','128Mb','256Mb'] 
plotdata = []
for i in range(4):
    valid = np.isfinite(t1s[i]) #* np.isfinite(t2s[i])
    subsetind = np.random.permutation(np.sum(valid))[:100000]
    plotdata.append(pd.DataFrame({'x':np.array(p1s[i])[valid][subsetind], 'y':np.array(t1s[i])[valid][subsetind], \
                                  'level': levels[i], 'cell': 'H1-ESC'}))
    plotdata.append(pd.DataFrame({'x':np.array(p2s[i])[valid][subsetind], 'y':np.array(t2s[i])[valid][subsetind], \
                                  'level': levels[i], 'cell': 'HFF'}))
    
plotdata = pd.concat(plotdata, axis=0)
plotdata['level'] = pd.Categorical(plotdata['level'], levels)
plotdata.to_csv('./figure_data/predeval.256m.crosschrom.inter.csv')
